In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
import json
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [37]:
response = requests.get("https://www.lesoir.be/18/sections/le-direct")
response.text

'<!doctype html>\n<html lang="fr" prefix="og: http://ogp.me/ns# fb: http://ogp.me/ns/fb# article: http://ogp.me/ns/article#">\n<head>\n  <meta http-equiv="Content-Type" content="text/html; charset=utf-8">\n  <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1"><script type="text/javascript">(window.NREUM||(NREUM={})).init={privacy:{cookies_enabled:true},ajax:{deny_list:["bam.eu01.nr-data.net"]},distributed_tracing:{enabled:true}};(window.NREUM||(NREUM={})).loader_config={agentID:"322533549",accountID:"2762144",trustKey:"2762144",xpid:"VgEBU1dXDBACVlVXBwAGVVY=",licenseKey:"NRJS-36410eda0fc8aae6e0f",applicationID:"224501132"};;/*! For license information please see nr-loader-spa-1.236.0.min.js.LICENSE.txt */\n(()=>{"use strict";var e,t,r={5763:(e,t,r)=>{r.d(t,{P_:()=>l,Mt:()=>g,C5:()=>s,DL:()=>v,OP:()=>T,lF:()=>D,Yu:()=>y,Dg:()=>h,CX:()=>c,GE:()=>b,sU:()=>_});var n=r(8632),i=r(9567);const o={beacon:n.ce.beacon,errorBeacon:n.ce.errorBeacon,licenseKey:void 0,applicationID:void 0,s

In [39]:
base_url = "https://www.lesoir.be/"

In [41]:
soup = bs(response.content, "html.parser")
articles = soup.select("h3 > a")
print(len(articles))
article_one = articles[1].get('href')
article_url = base_url + article_one
article_url



20


'https://www.lesoir.be//528944/article/2023-08-02/le-prix-de-lessence-va-continuer-daugmenter'

In [16]:
urls = soup.find_all("h3 a",attrs='href')
len(urls)

0